In [36]:
import pandas as pd

In [37]:
df = pd.read_csv('/content/final_movie_data.csv')

In [38]:
#lower
df['name'] = df['name'].str.lower()
df['description'] = df['description'].str.lower()

In [39]:
#remove html tags
import re
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [40]:
remove_html_tags(df['description'][0])
re

<module 're' from '/usr/lib/python3.10/re.py'>

In [41]:
df['description'] = df['description'].fillna('').apply(remove_html_tags)

In [42]:
df['description'] = df['description'].apply(remove_html_tags)


In [43]:
#remove urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

In [44]:
df['description'] = df['description'].apply(remove_urls)

In [45]:
#remove punctuation
import string
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [46]:
df['description'] = df['description'].apply(remove_punctuation)

In [47]:
#spelling correction
from textblob import TextBlob
def correct_spelling(text):
    return str(TextBlob(text).correct())

In [48]:
# df['description'] = df['description'].apply(correct_spelling)

In [49]:
from tqdm import tqdm

# Assuming df is your DataFrame and correct_spelling is your function
tqdm.pandas()

#df['description'] = df['description'].progress_apply(correct_spelling)


In [50]:
#remove stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
stop_words = set(stopwords.words('english'))

# Optimized function to remove stopwords
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

df['description'] = df['description'].apply(remove_stopwords)

In [52]:
#spacy tokenization
import spacy
nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [53]:
def spacy_tokenizer(sentence):
    mytokens = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in string.punctuation ]
    return mytokens


In [54]:
spacy_tokenizer(df['description'][0])

['imprison',
 '1940s',
 'double',
 'murder',
 'wife',
 'lover',
 'upstande',
 'banker',
 'andy',
 'dufresne',
 'begin',
 'new',
 'life',
 'shawshank',
 'prison',
 'put',
 'accounting',
 'skill',
 'work',
 'amoral',
 'warden',
 'long',
 'stretch',
 'prison',
 'dufresne',
 'come',
 'admire',
 'inmate',
 'include',
 'old',
 'prisoner',
 'name',
 'red',
 'integrity',
 'unquenchable',
 'sense',
 'hope']

In [55]:
#df['description'] = df['description'].apply(spacy_tokenizer)

In [61]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print(missing_values)

name                    0
description             0
original_description    0
dtype: int64


In [56]:
from tqdm import tqdm
tqdm.pandas()
df['description'] = df['description'].progress_apply(spacy_tokenizer)

100%|██████████| 9522/9522 [01:27<00:00, 108.83it/s]


In [69]:
import nltk
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import pandas as pd

# Download the WordNet data
nltk.download('wordnet')
nltk.download('punkt')

# Initialize the lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [70]:
# Function to lemmatize text
def lemmatize_text(text):
    if isinstance(text, str):  # Check if the text is a string
        # Tokenize the text
        words = nltk.word_tokenize(text)
        # Lemmatize each word
        return ' '.join(wordnet_lemmatizer.lemmatize(word) for word in words)
    else:
        return text  # Return the input unchanged if it's not a string


In [71]:
# Apply lemmatization to the 'description' column
tqdm.pandas()  # Initialize tqdm for progress_apply
df['description'] = df['description'].progress_apply(lemmatize_text)

100%|██████████| 9522/9522 [00:00<00:00, 693623.76it/s]


In [74]:
# Test the lemmatization function with an example sentence
sentence = "The leaves on the trees are falling rapidly."
lemmatized_sentence = lemmatize_text(sentence)

In [75]:
print("Original:", sentence)
print("Lemmatized:", lemmatized_sentence)

Original: The leaves on the trees are falling rapidly.
Lemmatized: The leaf on the tree are falling rapidly .
